Derick Hanscom

Dr. Ryan White

Homework #1

In [3]:
# Library used to process data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# Library used to calculate performance metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Library used to perform Linear Regression
from sklearn.linear_model import LinearRegression
# Library used to import desired files
from google.colab import files

##**Problem 1**


In [4]:
def normalize(columns):
  for i in columns:
    # Normalizes given columns according to the mean value in the column
    data[i] /= np.mean(data[i])

Using get_dummies() in the Pandas library, Columns with features of Yes/No were processed as binary numbers in seperate columns. Having both of these columns decreased the accuracy of the model thus one was removed. Next was to manually process the two columns House Style and Subdivision and map the data to assigned numbers. Next was to turn the List Price data into an integer by removing all commas and dollar signs. The next step was to assign the data to the input and output variables uses for calculations. After the variables were assigned a 20/80 testing/training split was used to randomly seperate the data.



In [5]:
# Allows user to upload file into google colab, Look under the text block to upload file
# File under inspection
data = pd.read_csv('Mount_Pleasant_Real_Estate_Data.csv')

# Uses get_dummies to turn all the columns with Yes/No features into binary numbers in seperate columns
data = pd.get_dummies(data, columns=['Duplex?', 'New Owned?', 'Has Pool?', 'Has Dock?', 'Fenced Yard',
                                     'Screened Porch?', 'Golf Course?', 'Fireplace?'])

# Drops all the added columns from get_dummies to improve accuracy, otherwise two columns with one being the inverse of the other
data = data.drop(columns=['Duplex?_Yes', 'New Owned?_Yes', 'Has Pool?_Yes', 'Has Dock?_Yes', 'Fenced Yard_Yes',
                          'Screened Porch?_Yes', 'Golf Course?_Yes', 'Fireplace?_Yes', 'Amenities', 'Misc Exterior',
                          'ID'])

# Maps the House Style column to a numbered representation of for each style
data['House Style'] = data['House Style'].map({'Townhouse': 0, 'Traditional': 1, 'Cottage': 2, 'Patio': 3,
                                               'Craftsman': 4, 'Ranch': 5, 'Charleston Single': 6, 'Colonial': 7,
                                               'Contemporary': 8, "Condominium": 9, 'Condo Regime': 10})

# Maps the Sub Division column to a numbered representation of for each location
data['Subdivision'] = data['Subdivision'].map({'Carolina Park': 0, 'Dunes West': 1, 'Park West': 2})

# Drops any columns with NA values, prevents errors with type conversions
data.dropna(inplace=True)

# Turns a price from a spreadsheet into an integer representation by removing dollar sign and comma
data['List Price'] = data['List Price'].str.replace('$', '', regex=True)
data['List Price'] = data['List Price'].str.replace(',', '', regex=True)
data['List Price'] = data['List Price'].astype(int)

# List of columns that will need to be normalized
columns = ['List Price', 'Bedrooms', 'Baths - Total', 'Baths - Full', 'Baths - Half', 'Stories', 'Square Footage', 'Year Built', 'Acreage', 'Covered Parking Spots', 'Number of Fireplaces']

# Calls the normalizing function to normalize colimns without Yes/No data types
normalize(columns)

# Sets the List Price into a variable y, these will be our target outputs
y = data['List Price'].to_numpy()

# Sets the remaining data excluding the list price as the X data
X = data.drop(columns=['List Price']).astype(float).to_numpy()
# Splits data into testing and training sets by a 20/80 percent split
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.2, random_state=1)

(245, 20)


## **Problem 2**

The next step is to instantiate the Linear Regression class from sklearn. This class fits a linear line with the least error between the data and the line. The parameters that minimize this error are then applied to the testing data to create testing predictions. These predictions are then compared to the true values where we can determine how well the model predicts.

In [ ]:
# Initializes Sklearns Linear Regression Class
model = LinearRegression()

# Uses the training data to determine theta parameters that minimize error
model.fit(trainX, trainY)

# Uses the fitted model to predict the training outputs
trainPredictions = model.predict(trainX)

# Uses the fitted model from the training data to predict the testing outputs
predictions = model.predict(testX)

Next we need to check how well the training set did and then compare with the results of the testing set. This will be used to determine how well the parameters from the model fit the testing. If the error in each data set is relatively close then the model is successful and the model did not under or over fit the data.

In [ ]:
print("Training Metrics")
# print the coefficient of determination r^2 for the training set
print('The r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics for training set
print('The mean squared error on the training set is', mean_squared_error(trainY, trainPredictions))
print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

print(f"\n")
print("Testing Metrics")
# print the coefficient of determination r^2 for the testing set
print('The r^2 score is', r2_score(testY, predictions))

# print quality metrics for testing set
print('The mean squared error on the test set is', mean_squared_error(testY, predictions))
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions))

Training Metrics
The r^2 score is 0.9150017079060816
The mean squared error on the training set is 0.020645511548928467
The mean absolute error on the training set is 0.10177031429334545


Testing Metrics
The r^2 score is 0.8818912275581665
The mean squared error on the test set is 0.026528940684087395
The mean absolute error on the test set is 0.10866616225110846


It seems that the hyper plane model fit quite well. Given how close the r^2 and the mean absolute error between the testing and training sets suggests that the model fit properly.

## **Problem 3**

Problem three fits a polynomial to the data and then performs linear regression to determine the weights that will best fit the regression model to the data. By setting fit_intercept to false we acknwoldge that the constant term has already been accounted for. The last step is to calculate the metrics and perform a comparison of the two.

In [9]:
# Takes in  largest desired degree of the polynomial
def univariatePolynomialBasis(m):
    def polynomialM(x):
        out = np.array([])
        for i in range(m + 1):
            # Adds a degree i term to the polynomial
            out = np.append(out, x ** i)
        return out

    # Returns the polynomial function
    return polynomialM

  # Calls the polynomial basis function and produces a degree 2 basis, quadratic
poly = univariatePolynomialBasis(2)

# Creates a polynomial for each feature for all data
Xh = np.apply_along_axis(poly, 1, trainX)
model = LinearRegression(fit_intercept=False)

print(Xh.shape)
print(trainX.shape)

# Fits all of the polynomials from the training data to the prices of the training data 
model.fit(Xh, trainY)
# Computes the training predictions using the theta values from fitted polynomial
trainPredictions = model.predict(Xh)

# Generates the predicted prices using a polynomial from the testing data and the theta values generated from the initial fitting
predictions = model.predict(np.apply_along_axis(poly,1,testX))

print("Training Metrics:")
# print the coefficient of determination r^2 for the training set
print('The r^2 score for training is', r2_score(trainY, trainPredictions))

# print quality metrics for the training set
print('The mean squared error on the training set is', mean_squared_error(trainY, trainPredictions))
print('The mean absolute error for training is', mean_absolute_error(trainY, trainPredictions))

print(f"\n")
print("Testing Metrics:")
# print the coefficient of determination r^2 for the testing set
print('The r^2 score for testing is', r2_score(testY, predictions))

# print quality metrics for the testing set
print('The mean squared error on the test set is', mean_squared_error(testY, predictions))
print('The mean absolute error for testing is', mean_absolute_error(testY, predictions))

AxisError: ignored

It seems that our model accurately predicts with the given data. The quadratic representation is slightly more accurate than the fitted hyperplane. The measure of likeness from the r^2 and the small amount of error in the MAE suggest that the model performed quite well. When comparing the r^2 and MAE between the testing and training sight it appears that our model was neither underfit nor overfit.

## **Problem #4**

$a)$

 Weighted Squared Sum Loss Function: Where P is a diagonal matrix with weights on the diagonal

$L(\theta)=\sum_{i=1}^np_i(\hat{f}(x_i)-y_i)^2=P(X\theta-y)^T(X\theta-y)=P(\theta^TX^T-y^T)(X\theta-y)=P(\theta^TX^TX\theta-\theta^TX^Ty-y^TX\theta+y^Ty)=P(\theta^TX^TX\theta-2\theta^TX^Ty-y^Ty)$


$b)$

 Next take derivative with repsect to theta and set to zero to solve for theta

$\frac{\partial L(\theta)}{\partial \theta}=(2X^TPX\theta-2X^TPy)$

$\frac{\partial L(\theta)}{\partial \theta}=0 => \hat{\theta}=(X^TPX)^{-1}X^TPy$

$c)$

A penalizing scheme that increases over time will cause occurences close to the start time to be more impactful in the loss function than occurences with less significance farther away from the start time. This can be used in a stock model, it makes sense because the information of the previous day will be more pertinant to the prediction for the next day than the information from a year ago.

**Problem 4 can also be solved by using the summation convention**

$L(\theta)=\sum_{i=1}^np_i(\hat{f}(x_i)-y_i)^2=\sum_{i=1}^np_i(x_i\theta_i-y_i)^2=\sum_{i=1}^np_ix_i^2\theta_i^2-2p_ix_i\theta_iy_i+p_iy_i^2$
$\frac{\partial L(\theta)}{\partial \theta}=\sum_{i=1}^n2p_i(x_i^2\theta_i-x_iy_i)$

$\frac{\partial L(\theta)}{\partial \theta}=0=>\hat{\theta}=\sum_{i=1}^ny_i/x_i$

##$Bonus$

We can make the columns of data named "Misc Exterior" and "Amenities" more useful by simply counting the number of additions that the house has. If the house comes furnished and comes with a porch and shed, the house will be of higher value. By counting the number of additions in each column, a number can be generated that roughly represents the extra pros of the house which will correlate with higher prices. Another idea would be to search for key words that correlate to higher prices and count the number of times those key words are used for each house. These changes would numerically influence the predictive nature of the model and could show to be useful.